# Generate shape coefficients

The following steps are performed to obtain all the artifacts for shape coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data for pressure coefficient

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CePathManager
import pathlib

path_manager = CePathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Ce config file

In [2]:
from cfdmod.use_cases.pressure.shape.Ce_config import CeCaseConfig

config_path = pathlib.Path("./fixtures/tests/pressure/Ce_params.yaml")
post_proc_cfg = CeCaseConfig.from_file(config_path)

post_proc_cfg

CeCaseConfig(shape_coefficient={'measurement_1': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 5.0, 10.0, 15.0, 20.0]), no_zoning=['t1_ym'], exclude=['p1_xp'], exceptions={'zoning1': ExceptionZoningModel(x_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0, 225.0, 250.0], y_intervals=[0.0, 25.0, 50.0, 75.0, 100.0, 125.0, 150.0, 175.0, 200.0], z_intervals=[0.0, 20.0], surfaces=['t2_yp'])}), statistics=['min', 'max', 'std', 'avg', 'skewness', 'kurtosis'], sets={'lanternim': ['L1_xp', 'L2_yp', 'L3_zp_yp', 'L4_zp_ym', 'L5_ym', 'L6_xm']}, transformation=GeometryTransformationConfig(translation=(0, 0, 0), rotation=(0.0, 0.0, 0.0), fixed_point=(0, 0, 0), scale=(1, 1, 1))), 'measurement_2': CeConfig(zoning=ZoningConfig(global_zoning=ZoningModel(x_intervals=[0.0, 250.0], y_intervals=[0.0, 200.0],

Read LNAS Mesh

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.normalized.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.name, mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

('galpao', (1549, 3), (2915, 3))

Read Pressure Coefficient data

In [4]:
import pandas as pd

cp_data_path = pathlib.Path("./fixtures/tests/pressure/data/cp_t.resampled.hdf")
cp_data = pd.read_hdf(cp_data_path)

cp_data_to_use = cp_data.to_frame() if isinstance(cp_data, pd.Series) else cp_data

Add region index to pressure coefficient data

In [5]:
from cfdmod.api.vtk.write_vtk import write_polydata, merge_polydata
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.shape.Ce_data import process_surface, get_surfaces_raw_data
from cfdmod.use_cases.pressure.geometry import get_excluded_surfaces, create_NaN_polydata

n_timesteps = cp_data_to_use["time_step"].unique().shape[0]

for cfg_label, cfg in post_proc_cfg.shape_coefficient.items():
    processed_polydata = []
    logger.info(f"Processing {cfg_label} ...")

    sfc_dict = {set_lbl: sfc_list for set_lbl, sfc_list in cfg.sets.items()}
    sfc_dict |= {sfc: [sfc] for sfc in mesh.surfaces.keys() if sfc not in cfg.surfaces_in_sets}

    data_columns = []
    surfaces_to_process = get_surfaces_raw_data(surface_dict=sfc_dict, cfg=cfg, mesh=mesh)
    for sfc_lbl, raw_surface in surfaces_to_process.items():
        logger.info(f"Processing surface {sfc_lbl}")

        processed_surface = process_surface(
            raw_surface=raw_surface,
            cfg=cfg,
            cp_data=cp_data_to_use,
            n_timesteps=n_timesteps,
        )
        processed_polydata.append(processed_surface.polydata)
        data_columns = processed_surface.surface_ce_stats.columns

        logger.info(f"Processed surface {sfc_lbl}")

    sfc_list = [sfc for sfc in cfg.zoning.exclude if sfc in mesh.surfaces.keys()]
    sfc_list += [
        sfc
        for set_lbl, sfc_set in cfg.sets.items()
        for sfc in sfc_set
        if set_lbl in cfg.zoning.exclude
    ]
    if len(sfc_list) != 0:
        excluded_sfcs = get_excluded_surfaces(mesh=mesh, sfc_list=sfc_list)
        excluded_sfcs.export_stl(path_manager.get_excluded_surface_path(cfg_label))
        # Include polydata with NaN values
        columns = [col for col in data_columns if col not in ["point_idx", "region_idx"]]
        excluded_polydata = create_NaN_polydata(mesh=excluded_sfcs, column_labels=columns)
        processed_polydata.append(excluded_polydata)

    # Output 5: VTK
    merged_polydata = merge_polydata(processed_polydata)
    write_polydata(path_manager.get_vtp_path(mesh.name, cfg_label), merged_polydata)

    logger.info(f"Merged polydata for {cfg_label}")

[2023-11-13 15:03:11,480] [INFO] - cfdmod - Processing measurement_1 ... (440998233.py:10)
[2023-11-13 15:03:11,483] [INFO] - cfdmod - Surface p1_xp ignored! (Ce_data.py:77)
[2023-11-13 15:03:11,486] [INFO] - cfdmod - Processing surface lanternim (440998233.py:18)
[2023-11-13 15:03:11,809] [INFO] - cfdmod - Processed surface lanternim (440998233.py:29)
[2023-11-13 15:03:11,810] [INFO] - cfdmod - Processing surface m1_yp (440998233.py:18)
[2023-11-13 15:03:11,865] [INFO] - cfdmod - Processed surface m1_yp (440998233.py:29)
[2023-11-13 15:03:11,866] [INFO] - cfdmod - Processing surface m2_zp (440998233.py:18)
[2023-11-13 15:03:11,930] [INFO] - cfdmod - Processed surface m2_zp (440998233.py:29)
[2023-11-13 15:03:11,930] [INFO] - cfdmod - Processing surface m3_zm (440998233.py:18)
[2023-11-13 15:03:12,003] [INFO] - cfdmod - Processed surface m3_zm (440998233.py:29)
[2023-11-13 15:03:12,004] [INFO] - cfdmod - Processing surface p2_xp (440998233.py:18)
[2023-11-13 15:03:12,052] [WARNING] - c